In [3]:
import torch
import torch.nn.functional as F
from torch import nn
import timeit

kernel_size = 80  # 10
stride = 4  # 2


def m3_forward(data = torch.rand(1, 1024)):
    conv1 = nn.Conv1d(
        in_channels=1,
        out_channels=256,
        kernel_size=kernel_size,
        stride=stride
    )
    out1 = conv1(data)

    bn1 = nn.BatchNorm1d(256)
    out2 = F.relu(bn1(out1.unsqueeze(0)))

    pool1 = nn.MaxPool1d(4)
    out2 = pool1(out2)

    conv2 = nn.Conv1d(256, 256, kernel_size=3)
    bn2 = nn.BatchNorm1d(256)
    out3 = conv2(out2)
    out3 = F.relu(bn2(out3))

    out3 = F.avg_pool1d(out2, out2.shape[-1])

    fc1 = nn.Linear(256, 2)
    sigmoid = nn.Sigmoid()

    output = fc1(out3.permute(0, 2, 1))
    output = sigmoid(output)


x = torch.rand(1, 1024)

t0 = timeit.Timer(
    stmt='m3_forward(x)',
    setup='from __main__ import m3_forward',
    globals={'x': x})

print(f'm3_forward:  {t0.timeit(100) / 100 * 1e6:>5.1f} us')

m3_forward:  2932.7 us


In [4]:
import torch.utils.benchmark as benchmark

t0 = benchmark.Timer(
    stmt='m3_forward(x)',
    setup='from __main__ import m3_forward',
    globals={'x': x})

print(t0.timeit(100))

m3_forward(x)
setup: from __main__ import m3_forward
  3.49 ms
  1 measurement, 100 runs , 1 thread
